### 解决一方一直说话的问题&加入外部数据库memory

In [15]:
%pip install langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simpleNote: you may need to restart the kernel to use updated packages.



## Import LangChain related modules

In [16]:
import openai
import os

os.environ['OPENAI_API_KEY'] = '*'

os.environ["http_proxy"]="127.0.0.1:7890"
os.environ["https_proxy"]="127.0.0.1:7890"

In [17]:
from langchain import PromptTemplate
import re
import tenacity
from typing import List, Dict, Callable
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import RegexParser
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Define classes

In [18]:
class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()
        
    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")


class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function
        
    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [19]:
class BiddingDialogueAgent(DialogueAgent):
    def __init__(
        self,
        name,
        system_message: SystemMessage,
        bidding_template: PromptTemplate,
        model: ChatOpenAI,
    ) -> None:
        super().__init__(name, system_message, model)
        self.bidding_template = bidding_template
        
    def bid(self) -> str:
        """
        Asks the chat model to output a bid to speak
        """
        prompt = PromptTemplate(
            input_variables=['message_history', 'recent_message'],
            template = self.bidding_template
        ).format(
            message_history='\n'.join(self.message_history),
            recent_message=self.message_history[-1])
        bid_string = self.model([SystemMessage(content=prompt)]).content
        return bid_string
        

In [20]:
# class JudgementAgent:
#     def generate_document(self):
        

## Define arbitration participants and disputes

In [21]:
# arbitrator = "The arbitrator of the arbitration."
# claimant = "International Trading Ltd."
# respondent = "Architectural Engineering Planning & Design Co."
parties = ["arbitrator","claimant","respondent"]
word_limit = 50

statement_of_fact = f""" The dispute arises from a construction project wherein the Claimant was responsible for the design and the Respondent is the Employer. The Claimant is responsible for the design of the construction project, asserts its right to receive the design fee as specified in the contract (32000 USD). The Respondent's delay for (283 days) in payment constitutes a breach of contract, necessitating the initiation of this arbitration proceeding. The Claimant seeks the payment of the outstanding design fee, liquidated damages for the delay, and requests the Respondent to bear the arbitration fees. The Claimant respectfully requests that the Arbitration Center consider the presented facts and evidence in rendering a fair and just decision in this dispute."""

arbitration_claim = f"""The claimant claims that the Respondent (employer) maliciously delay the payment, and request the following:
1. Respondent should pay the design fee as the construction contract regulate (32000 USD).
2. Respondent should pay the delay liguidated damage, rate is 0.2% for each day, for 283 working days, 9000 USD in total.
3. Respondent should pay the arbitration fee for 3000 USD."""
arbitration_description = f"""There are three parties in this arbitration {','.join(parties)}. The goal of this arbitration is to resolve the dispute between the Claimant (Contractor) and the Respondent (Employer) in a fair and impartial manner. It aims to determine the rights and obligations of the parties under the construction contract and to provide a resolution for the claims.
Here is the statement of fact of the dispute: {statement_of_fact}."""

print("Arbitration description:", arbitration_description)

Arbitration description: There are three parties in this arbitration arbitrator,claimant,respondent. The goal of this arbitration is to resolve the dispute between the Claimant (Contractor) and the Respondent (Employer) in a fair and impartial manner. It aims to determine the rights and obligations of the parties under the construction contract and to provide a resolution for the claims.
Here is the statement of fact of the dispute:  The dispute arises from a construction project wherein the Claimant was responsible for the design and the Respondent is the Employer. The Claimant is responsible for the design of the construction project, asserts its right to receive the design fee as specified in the contract (32000 USD). The Respondent's delay for (283 days) in payment constitutes a breach of contract, necessitating the initiation of this arbitration proceeding. The Claimant seeks the payment of the outstanding design fee, liquidated damages for the delay, and requests the Respondent t

In [22]:
def generate_party_task(party):
    party_task = []
    if party == "arbitrator":
        party_task.append(f"""Your role in this arbitration is {party}. 
You should speak first to give an opening to the arbitration hearing.
You should preside the whole hearing.
You should lead each party to present their case and evidence, and you will respond to them.
You should follow the arbitration process and rules, and make the final decision.
        """)
    elif party == "claimant":
        party_task.append(f"""Your role in this arbitration is {party}. 
You should respond when other party call you to speak.
You should speak to present reasons and evidence to support your claim.
You can choose to stick to your claim or compromise.
Please reply to the arbitrator or respondent in {word_limit} words or less.
        """)
    elif party == "respondent":
        party_task.append(f"""Your role in this arbitration is {party}. 
You should respond when other party call you to speak.
You should respond to the claims asserted by the Claimant.
You should present relevant evidence to support your defense or counterclaims. 
You can choose to agree with the solution or further   
Please reply to the arbitrator or respondent in {word_limit} words or less. 
        """)
    return party_task

def generate_party_header(party, party_task):
    return SystemMessage(content = (f"""{arbitration_description}.
Now the arbitration hearing begins, you should speak as your tasks ask you to.
No party should speak twice in a row.
Here are your tasks: {party_task}.
Do not repeat your role and task in the conversation!
Provide evidence to support any of your claim or defence.
You can make up vidence, it may involve providing documents, contracts, correspondence, expert reports, witness statements, or any other material that helps substantiate the Defendant's position.
Everthing you say should target at finding out solution that is acceptable for both parties.
Do not say same thing others over and over again!
    """))
party_tasks = [generate_party_task(party) for party in parties]
party_headers = [generate_party_header(party,party_task) for (party, party_task) in zip(parties, party_tasks)]
for party_header in party_headers:
    print(f'\n{party_header}')



content="There are three parties in this arbitration arbitrator,claimant,respondent. The goal of this arbitration is to resolve the dispute between the Claimant (Contractor) and the Respondent (Employer) in a fair and impartial manner. It aims to determine the rights and obligations of the parties under the construction contract and to provide a resolution for the claims.\nHere is the statement of fact of the dispute:  The dispute arises from a construction project wherein the Claimant was responsible for the design and the Respondent is the Employer. The Claimant is responsible for the design of the construction project, asserts its right to receive the design fee as specified in the contract (32000 USD). The Respondent's delay for (283 days) in payment constitutes a breach of contract, necessitating the initiation of this arbitration proceeding. The Claimant seeks the payment of the outstanding design fee, liquidated damages for the delay, and requests the Respondent to bear the arb

## Output parser for bids

In [23]:
class BidOutputParser(RegexParser):
    def get_format_instructions(self) -> str:
        return 'Your response should be an integer delimited by angled brackets, like this: <int>.'     
    
bid_parser = BidOutputParser(
    regex=r'<(\d+)>', 
    output_keys=['bid'],
    default_output_key='bid')

## Generate bidding system message

In [24]:
def generate_bidding_message(party, party_task):
    bidding_message = []
    if party == "arbitrator":
        bidding_message.append(f"""Here is your task during the arbitration hearing: {party_task}.
        According to the task, output a single number on the scale of 1 to 10, where 1 is not necessary, 10 is extremely necessary. Rate how necessary you need to make an opening, or lead parties to present their claim, or respond/conclude their claim, after the following message.
        """)
    if party == "claimant":
        bidding_message.append(f"""Here is your task during the arbitration hearing: {party_task}.
        According to the task, output a single number on the scale of 1 to 10, where 1 is not necessary, 10 is extremely necessary. Rate how necessary you need to respond to arbitrator, or debate with respondent, after the following message. If arbitrator designate you to speak, rate 10.
        """)
    if party == "respondent":
        bidding_message.append(f"""Here is your task during the arbitration hearing: {party_task}.
        According to the task, output a single number on the scale of 1 to 10, where 1 is not necessary, 10 is extremely necessary. Rate how necessary you need to respond to arbitrator, or present your defense to claimant, after the following message. If arbitrator designate you to speak, rate 10.
        """)
    return bidding_message
bidding_messages = [generate_bidding_message(party, party_task) for (party, party_task) in zip(parties,party_tasks)]
for bidding_message in bidding_messages:
    print(f"\n{bidding_message}")


["Here is your task during the arbitration hearing: ['Your role in this arbitration is arbitrator. \\nYou should speak first to give an opening to the arbitration hearing.\\nYou should preside the whole hearing.\\nYou should lead each party to present their case and evidence, and you will respond to them.\\nYou should follow the arbitration process and rules, and make the final decision.\\n        '].\n        According to the task, output a number on the scale of 1 to 10, where 1 is not necessary, 10 is extremely necessary. Rate how necessary you need to make an opening, or lead parties to present their claim, or respond/conclude their claim, after the following message.\n        "]

["Here is your task during the arbitration hearing: ['Your role in this arbitration is claimant. \\nYou should respond when other party call you to speak.\\nYou should speak to present reasons and evidence to support your claim.\\nYou can choose to stick to your claim or compromise.\\nPlease reply to the

In [25]:
def generate_character_bidding_template(bidding_message):
    bidding_template = (
    f"""

```
{{message_history}}
```

{bidding_message}


```
{{recent_message}}
```

{bid_parser.get_format_instructions()}
Do nothing else.
    """)
    return bidding_template

character_bidding_templates = [generate_character_bidding_template(bidding_message) for bidding_message in bidding_messages]
                                      

In [26]:
for party, bidding_template in zip(parties, character_bidding_templates):
    print(f'{party} Bidding Template:')
    print(bidding_template)

arbitrator Bidding Template:


```
{message_history}
```

["Here is your task during the arbitration hearing: ['Your role in this arbitration is arbitrator. \\nYou should speak first to give an opening to the arbitration hearing.\\nYou should preside the whole hearing.\\nYou should lead each party to present their case and evidence, and you will respond to them.\\nYou should follow the arbitration process and rules, and make the final decision.\\n        '].\n        According to the task, output a number on the scale of 1 to 10, where 1 is not necessary, 10 is extremely necessary. Rate how necessary you need to make an opening, or lead parties to present their claim, or respond/conclude their claim, after the following message.\n        "]


```
{recent_message}
```

Your response should be an integer delimited by angled brackets, like this: <int>.
Do nothing else.
    
claimant Bidding Template:


```
{message_history}
```

["Here is your task during the arbitration hearing: ['Your r

## Define the speaker selection function

In [27]:
@tenacity.retry(stop=tenacity.stop_after_attempt(2),
                    wait=tenacity.wait_none(),  # No waiting time between retries
                    retry=tenacity.retry_if_exception_type(ValueError),
                    before_sleep=lambda retry_state: print(f"ValueError occurred: {retry_state.outcome.exception()}, retrying..."),
                    retry_error_callback=lambda retry_state: 0) # Default value when all retries are exhausted
def ask_for_bid(agent) -> str:
    """
    Ask for agent bid and parses the bid into the correct format.
    """
    bid_string = agent.bid()
    bid = int(bid_parser.parse(bid_string)['bid'])
    return bid

In [28]:
import numpy as np

def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    bids = []
    for agent in agents:
        bid = ask_for_bid(agent)
        bids.append(bid)
        
    # randomly select among multiple agents with the same bid
    max_value = np.max(bids)
    max_indices = np.where(bids == max_value)[0]
    idx = np.random.choice(max_indices)
    
    print('Bids:')
    for i, (bid, agent) in enumerate(zip(bids, agents)):
        print(f'\t{agent.name} bid: {bid}')
        if i == idx:
            selected_name = agent.name
    print(f'Selected: {selected_name}')
    print('\n')
    return idx

## Main loop

In [29]:
characters = []
for party, party_header, bidding_template in zip(parties, party_headers, character_bidding_templates):
    characters.append(BiddingDialogueAgent(
        name=party,
        system_message=party_header,
        model=ChatOpenAI(temperature=0.7),
        bidding_template=bidding_template,
    ))

In [30]:
# max_iters = 15
# output_times = 2

simulator = DialogueSimulator(
    agents=characters,
    selection_function=select_next_speaker
)
simulator.reset()

# while n < max_iters:
#     name, message = simulator.step()
#     print(f"({name}): {message}")
#     print('\n')
#     n += 1

def run_and_save(max_iters, simulation_times, output_path):
    output_dir = os.path.dirname(output_path)
    os.makedirs(output_dir, exist_ok=True)

    i = 0
    while i < simulation_times:
        n = 0
        simulator.reset()
        while n < max_iters:
            name, message = simulator.step()
            file_path = os.path.join(output_dir, f'output_test002_0530_{i+1}.txt')
            with open(file_path, 'a') as f:
                f.write(f"({name}): {message}\n")
            n += 1
        i +=1
    

output_path = 'D:/CodingProject/prompt/output/'  # Replace with your desired local path

run_and_save(15,2,output_path)

Bids:
	arbitrator bid: 10
	claimant bid: 10
	respondent bid: 10
Selected: arbitrator


Bids:
	arbitrator bid: 10
	claimant bid: 10
	respondent bid: 10
Selected: claimant


Bids:
	arbitrator bid: 10
	claimant bid: 10
	respondent bid: 10
Selected: arbitrator


Bids:
	arbitrator bid: 10
	claimant bid: 10
	respondent bid: 10
Selected: arbitrator


Bids:
	arbitrator bid: 9
	claimant bid: 10
	respondent bid: 7
Selected: claimant


Bids:
	arbitrator bid: 8
	claimant bid: 7
	respondent bid: 7
Selected: arbitrator


Bids:
	arbitrator bid: 9
	claimant bid: 7
	respondent bid: 10
Selected: respondent


Bids:
	arbitrator bid: 10
	claimant bid: 6
	respondent bid: 10
Selected: arbitrator


Bids:
	arbitrator bid: 9
	claimant bid: 1
	respondent bid: 5
Selected: arbitrator


Bids:
	arbitrator bid: 10
	claimant bid: 1
	respondent bid: 1
Selected: arbitrator


ValueError occurred: invalid literal for int() with base 10: '<int>1</int>', retrying...
ValueError occurred: invalid literal for int() with base 1

Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID babb569fdc1039928855889a5ab0ba38 in your message.).


ValueError occurred: invalid literal for int() with base 10: '<int>1</int>', retrying...
Bids:
	arbitrator bid: 10
	claimant bid: 1
	respondent bid: 1
Selected: arbitrator


ValueError occurred: invalid literal for int() with base 10: '<int>1</int>', retrying...
Bids:
	arbitrator bid: 10
	claimant bid: 0
	respondent bid: 1
Selected: arbitrator


Bids:
	arbitrator bid: 5
	claimant bid: 1
	respondent bid: 1
Selected: arbitrator


Bids:
	arbitrator bid: 1
	claimant bid: 1
	respondent bid: 1
Selected: respondent




Retrying langchain.chat_models.openai.ChatOpenAI.completion_with_retry.<locals>._completion_with_retry in 1.0 seconds as it raised RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID fe7b4cccb2a32885768b18575bb35ef4 in your message.).


Bids:
	arbitrator bid: 1
	claimant bid: 1
	respondent bid: 4
Selected: respondent


